# Notebook for initial data exploration

In [2]:
import pandas as pd
import numpy as np

# Load the csv data from https://www.kaggle.com/datasets/flashgordon/usa-airport-dataset
df = pd.read_csv('../data/raw/Airports2.csv')

In [3]:
# Display the first few rows
print(df.head())


  Origin_airport Destination_airport    Origin_city Destination_city  \
0            MHK                 AMW  Manhattan, KS         Ames, IA   
1            EUG                 RDM     Eugene, OR         Bend, OR   
2            EUG                 RDM     Eugene, OR         Bend, OR   
3            EUG                 RDM     Eugene, OR         Bend, OR   
4            MFR                 RDM    Medford, OR         Bend, OR   

   Passengers  Seats  Flights  Distance    Fly_date  Origin_population  \
0          21     30        1       254  2008-10-01             122049   
1          41    396       22       103  1990-11-01             284093   
2          88    342       19       103  1990-12-01             284093   
3          11     72        4       103  1990-10-01             284093   
4           0     18        1       156  1990-02-01             147300   

   Destination_population  Org_airport_lat  Org_airport_long  \
0                   86219        39.140999        -96.6707

Below I caught that the Fly_date column is not in the datetime dataformate which we will need for proper calculations

In [4]:
# Get summary information about the dataframe
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606803 entries, 0 to 3606802
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Origin_airport          object 
 1   Destination_airport     object 
 2   Origin_city             object 
 3   Destination_city        object 
 4   Passengers              int64  
 5   Seats                   int64  
 6   Flights                 int64  
 7   Distance                int64  
 8   Fly_date                object 
 9   Origin_population       int64  
 10  Destination_population  int64  
 11  Org_airport_lat         float64
 12  Org_airport_long        float64
 13  Dest_airport_lat        float64
 14  Dest_airport_long       float64
dtypes: float64(4), int64(6), object(5)
memory usage: 412.8+ MB
None


In [5]:
# Convert the 'Fly_date' column to datetime type (assuming it's named 'Fly_date')
df['Fly_date'] = pd.to_datetime(df['Fly_date'])

# Check the data types again
print(df.dtypes)

Origin_airport                    object
Destination_airport               object
Origin_city                       object
Destination_city                  object
Passengers                         int64
Seats                              int64
Flights                            int64
Distance                           int64
Fly_date                  datetime64[ns]
Origin_population                  int64
Destination_population             int64
Org_airport_lat                  float64
Org_airport_long                 float64
Dest_airport_lat                 float64
Dest_airport_long                float64
dtype: object


Now I am going to modify the date information by creating new columns, this will help make the information more usable down the line, but if I was working in an actual pipeline, it would be most cost effective to make sure this data is captured correctly further up stream.

In [6]:
import holidays

df['Year'] = df['Fly_date'].dt.year
df['Month'] = df['Fly_date'].dt.month
df['Day_of_Week'] = df['Fly_date'].dt.dayofweek
df['Day_of_Month'] = df['Fly_date'].dt.day
df['Quarter'] = df['Fly_date'].dt.quarter
reference_date = pd.Timestamp('1990-01-01')
df['Days_Since_Reference'] = (df['Fly_date'] - reference_date).dt.days
df['Month_Sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_Cos'] = np.cos(2 * np.pi * df['Month'] / 12)
us_holidays = holidays.US()
df['Is_Holiday'] = df['Fly_date'].isin(us_holidays).astype(int)

# Function to assign seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Month'].apply(get_season)
df = df.drop('Fly_date', axis=1)

Below I caught that there are null values for lat and long so I need to explore that part of the data deeper

In [7]:
# Check for missing values
print(df.isnull().sum())

Origin_airport               0
Destination_airport          0
Origin_city                  0
Destination_city             0
Passengers                   0
Seats                        0
Flights                      0
Distance                     0
Origin_population            0
Destination_population       0
Org_airport_lat           6954
Org_airport_long          6954
Dest_airport_lat          6807
Dest_airport_long         6807
Year                         0
Month                        0
Day_of_Week                  0
Day_of_Month                 0
Quarter                      0
Days_Since_Reference         0
Month_Sin                    0
Month_Cos                    0
Is_Holiday                   0
Season                       0
dtype: int64


In [8]:
# Filter the DataFrame for null latitude and longitude values
null_lat_long = df[df[['Org_airport_lat', 'Org_airport_long', 'Dest_airport_lat', 'Dest_airport_long']].isnull().any(axis=1)]

# Display the filtered DataFrame
print(null_lat_long)

        Origin_airport Destination_airport      Origin_city  \
0                  MHK                 AMW    Manhattan, KS   
945                RBG                 RDM     Roseburg, OR   
946                RBG                 RDM     Roseburg, OR   
947                RBG                 RDM     Roseburg, OR   
948                RBG                 RDM     Roseburg, OR   
...                ...                 ...              ...   
3595391            SCF                 PBI      Phoenix, AZ   
3603513            RND                 COS  San Antonio, TX   
3604641            HIK                 COS     Honolulu, HI   
3606576            YIP                 PHD      Detroit, MI   
3606802            FWA                 OH1   Fort Wayne, IN   

                   Destination_city  Passengers  Seats  Flights  Distance  \
0                          Ames, IA          21     30        1       254   
945                        Bend, OR           0      0        1       131   
946         

In [9]:
# Get the number of rows
num_rows = df.shape[0]

# Print the number of rows
print(f'The CSV file has {num_rows} rows.')

The CSV file has 3606803 rows.


Since the null values are a small percentage of the total rows (0.004%) I am going to drop them.

In [10]:
# Drop rows with null values in specified latitude and longitude columns
df_cleaned = df.dropna(subset=['Org_airport_lat', 'Org_airport_long', 'Dest_airport_lat', 'Dest_airport_long'])

# Display the cleaned DataFrame
print(df_cleaned)

# Optionally, check how many rows were dropped
print(f'Number of rows dropped: {len(df) - len(df_cleaned)}')

        Origin_airport Destination_airport         Origin_city  \
1                  EUG                 RDM          Eugene, OR   
2                  EUG                 RDM          Eugene, OR   
3                  EUG                 RDM          Eugene, OR   
4                  MFR                 RDM         Medford, OR   
5                  MFR                 RDM         Medford, OR   
...                ...                 ...                 ...   
3606797            STL                 TBN       St. Louis, MO   
3606798            STL                 TBN       St. Louis, MO   
3606799            STL                 TBN       St. Louis, MO   
3606800            STL                 TBN       St. Louis, MO   
3606801            CGI                 TBN  Cape Girardeau, MO   

              Destination_city  Passengers  Seats  Flights  Distance  \
1                     Bend, OR          41    396       22       103   
2                     Bend, OR          88    342       19     

In [11]:
# last check for missing values
print(df_cleaned.isnull().sum())

Origin_airport            0
Destination_airport       0
Origin_city               0
Destination_city          0
Passengers                0
Seats                     0
Flights                   0
Distance                  0
Origin_population         0
Destination_population    0
Org_airport_lat           0
Org_airport_long          0
Dest_airport_lat          0
Dest_airport_long         0
Year                      0
Month                     0
Day_of_Week               0
Day_of_Month              0
Quarter                   0
Days_Since_Reference      0
Month_Sin                 0
Month_Cos                 0
Is_Holiday                0
Season                    0
dtype: int64


Now I am going to save the cleaned data to a sqlite database in the processed folder

In [12]:
import os
from sqlalchemy import create_engine

# Specify the path to your processed data folder
processed_data_folder = '../data/processed'

# Ensure the folder exists
if not os.path.exists(processed_data_folder):
    os.makedirs(processed_data_folder)

# Define the path to the SQLite database file
sqlite_db_path = os.path.join(processed_data_folder, 'cleaned_data.db')

In [13]:
# Create an SQLite engine that connects to the database
engine = create_engine(f'sqlite:///{sqlite_db_path}')

In [14]:
# Save the cleaned DataFrame to an SQLite table
df_cleaned.to_sql('cleaned_flight_data', engine, if_exists='replace', index=False)

# Confirm the process
print(f"Cleaned data has been saved to the SQLite database at: {sqlite_db_path}")

Cleaned data has been saved to the SQLite database at: ../data/processed\cleaned_data.db


In [15]:
# Query the saved data from the SQLite table
df_from_sql = pd.read_sql('cleaned_flight_data', engine)

# Display the retrieved data
print(df_from_sql.head())

  Origin_airport Destination_airport  Origin_city Destination_city  \
0            EUG                 RDM   Eugene, OR         Bend, OR   
1            EUG                 RDM   Eugene, OR         Bend, OR   
2            EUG                 RDM   Eugene, OR         Bend, OR   
3            MFR                 RDM  Medford, OR         Bend, OR   
4            MFR                 RDM  Medford, OR         Bend, OR   

   Passengers  Seats  Flights  Distance  Origin_population  \
0          41    396       22       103             284093   
1          88    342       19       103             284093   
2          11     72        4       103             284093   
3           0     18        1       156             147300   
4          11     18        1       156             147300   

   Destination_population  ...  Year  Month  Day_of_Week  Day_of_Month  \
0                   76034  ...  1990     11            3             1   
1                   76034  ...  1990     12            5  